In [ ]:
##final code for paper 1 (corrected, 2023-04-19), non-dimensional form, Papper 4, section 4.2 (with inflow(Robin) at left) 
from numpy import * 
from pylab import *
from scipy.optimize import curve_fit
import scipy.sparse  
from scipy.sparse import diags 
from scipy.integrate import odeint 
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit # to fit the data in some curve
from itertools import cycle # need in for loop to plot with different dotted lines
import time 
start_time = time.time()

#parameter
D = 1 
beta = 500# 
H = 2.5 # Henry's constant (dimensionless)
#### parameter based on experimental
S_0 = 0.01 #initial penetration height
x_ref = 10 ##characterstic length x_ref
####
a_0 = 500
m_0 = 0.5
m_ref = m_0

### constraints paramter 
b = 10

#dimensionless number
A_0 = a_0*m_ref*x_ref/D
Bi = beta*x_ref/D
#######

h_0 = S_0/x_ref
####time #####

deltaT = 0.001 # timestep in dimension form
t_ref = x_ref**2/D # time scale

Tmax = 0.0001 #dimensionlless form
dt_f = 5e-9 #fem time step size in dimensionlless form

#total no of time nodes
T= int(ceil(Tmax/dt_f))

#we want to plot u solution  at these time in both RWM and FEM to compare 
T_dim = [round(number/(dt_f)) for number in [0,Tmax/4,Tmax/2, 3*Tmax/4]] 
print(T_dim)

###space discretization in dimensionless form
N = 101
x = linspace(0,1,N) 
h = 1/(N -1) 

##### tridiagonal matrix for M time derivative 
M = diags([h/6, 2*h/3, h/6], [-1, 0, 1], shape=(N, N)).toarray() 
M[0,0] = M[0, 0]/2 
M[N-1, N-1] =  M[N-1, N-1]/2 
Min = linalg.inv(M)  #inverse of M

#######tridiagonal matrix for K (grad(u).grad(phi)) 
K = diags([-1/h, 2/h, -1/h], [-1, 0, 1], shape=(N, N)).toarray() 
K[0,0] = K[0, 0]/2 
K[N-1, N-1] =  K[N-1, N-1]/2 

######## tridigonal matrix for mixed term  
diagonals = zeros((3, N))   # 3 diagonals 
for i in range(1, N-1): 
    diagonals[1,i] = -h/3  
diagonals[1, 0] = -1/6*(h +3*x[0]) #h/3 - x[1]/2  
diagonals[1, N-1] =  1/6*(3*x[N-1]-h) #h/3 - x[N-2]/2     
for i in range(N-1):     
    diagonals[2,i+1] = x[i+1]/2 - h/3 
for i in range(N-1):     
    diagonals[0,i] = -(x[i]/2 + h/3)   
k = array([diagonals[0,0:N-1], diagonals[1], diagonals[2,1:]], dtype=object) 
A = diags(k,[-1,0,1]).toarray() 

#vectors e_1 and e_{N} (it is needed for boundary terms)
E1 = N*[0] 
E1[0] = 1
EN = N*[0]
EN[N-1] = 1
In = [0.5]

for alpha in In:
    def Model(u, t): 
        dwdt = A_0*(u[N-1] - alpha/x_ref*u[N]/m_ref) 
        dudt = (dwdt/u[N])*dot(dot(Min, A), u[0:N]) - (1/(u[N])**2)*dot(dot(Min, K), u[0:N]) + (1/u[N])*Bi*(b/m_ref - H*u[0])*dot(Min,E1) - (dwdt/u[N])*u[N-1]*dot(Min,EN)   
        dudt = list(dudt) 
        dudt.append(dwdt) 
        return dudt
    u0 = (N)*[m_0/m_ref] 
    u0.append(h_0) 

    #time discretization
    t = linspace(0, Tmax, T) 
    
    #solve ODE 
    z = odeint(Model, u0, t) 
    m = z[:,:]

   #extracting the data of moving boundary
    hh=z[:,-1]
    
    ######trasnfering back to moving doamin 
    md = np.multiply(x,np.transpose([hh])) #moving domain over time 
    mm = np.multiply(m[:,:-1],np.transpose([hh])) #we need for computing the mass
    
    #for plotting purpose (plotting on dimensionless form)
    plt.figure()
    plt.plot(md[T_dim[0], :], m[T_dim[0], :-1], linewidth=2) 
    plt.plot(md[T_dim[1], :], m[T_dim[1], :-1], linewidth=2) 
    plt.plot(md[T_dim[2], :], m[T_dim[2], :-1], linewidth=2) 
    plt.plot(md[T_dim[3], :], m[T_dim[3], :-1], linewidth=2) 
    plt.grid()
    plt.show()
    
    #ploting in dimensional variable
    t_1 = t*t_ref #time nodes in moving domain t = tau*t_re
    t_1 = t #for plotting in non-dimensional form (comment this if you want to plot in dimensional form)
    St = x_ref*hh #z[:,N] # the position of moving boundary s(t) = x_ref*h(tau)
    St = hh #z[:,N] # the position of moving boundary s(t) = x_ref*h(tau), comment this if you want to plot in dimensional form

    ########### ploting in the dimensional form (if needed) ######
    #md_d = md*x_ref #moving lenght (domain) in dimensional form)
    #md_d = md #moving lenght (domain) in non-dimensional form)
    #plt.figure()
    #plt.plot(md_d[T_dim[0],:], m[T_dim[0],:-1], linewidth=2)
    #plt.plot(md_d[T_dim[1],:], m[T_dim[1],:-1], linewidth=2)
    #plt.plot(md_d[T_dim[2],:], m[T_dim[2],:-1], linewidth=2)
    #plt.plot(md_d[T_dim[3],:], m[T_dim[3],:-1], linewidth=2)
    #plt.grid()
    #plt.show()
    ##########
    
    plt.figure()
    plt.plot(t, hh, linewidth=2) 
    plt.legend([r"FE approximation"], loc='upper left') 
    plt.xlabel(r'$\tau$')
    plt.ylabel(r'$h(\tau)$')
    plt.grid()
    plt.show()
    
    ####calculating the number of walker over time 
    Total_mass_over_time = mm[:, :].sum(axis=1)
    print('total_walker in time=',Total_mass_over_time[: -1])
    plt.figure()
    plt.plot(t, h*Total_mass_over_time[:])
    plt.xlabel(r'$\tau$')
    plt.ylabel(r'Total mass of concentration $M(\tau)$')
    plt.show()
end_time = time.time()
print('computation time=', (end_time-start_time)/60, 'minute(s)')
   

In [ ]:
#RWM for my problem dimensionless form (paper section 4.2, Robin,s Bc at left) 2023-10-09
#T means u and s means h (based on the paper)

#%matplotlib inline
from math import ceil, floor, erf
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import random
import numpy as np
import time
start_time = time.time()

L=0.045 # Length of domain

########## parameters in my problem
D = 1
beta = 500
H = 2.5

####  parameter based on experimental
S_0 = 0.01
x_ref = 10

####
a_0 = 500
m_0 = 0.5
m_ref = m_0

### constraints paramter 
t_ref = x_ref**2/D
b = 10
#dimensionless number
A_0 = a_0*m_ref*x_ref/D
#print('A_0=', A_0)
Bi = beta*x_ref/D
#print('Biot=', Bi)
########## 
t_max= 0.0001 # Maximum time
#nn= [1000, 1500, 2000]# Number of walkers.
n =100
K = 0.5 # this constant we define for sigma, for example, sigma(h(tau)) = 5*h(tau)
time_refine = 0.5
#Dt = [5e-08]#, 5e-08/time_refine]
dt = 5e-08*time_refine
dx = np.sqrt(2*D*dt) #dx is chosen in this way for RWM

# number to filter the data from finite element data because we solved for different timestep, dt_f is used in FEM
NN = int(dt/dt_f) 

# Number of points in the spatial- and time-domains.
N_x=ceil(L/dx) 
N_t=ceil(t_max/dt) 

# Matrix representing T(x,t), initially set to 0.
T=np.zeros((N_x, N_t), dtype=int)
s_vector=np.zeros(N_t)# Vector representing the moving boundary s(t).

#initialize the increment of moving boundary ds
ds = np.zeros(N_t)
ds[0] = dt*A_0*(1 - K*S_0/m_0)
s= S_0/x_ref #initial value (h_0 in paper)

j_t=0
j_s=floor(s/dx) # Indices for time and the position of the moving boundary s(t).

while j_t < N_t-1 and j_s < N_x-1:
    if j_t == 0: 
        for j in range(j_s):
            T[j, 0] = 1*n
    else: 
        #T[0, j_t] = round(((n*dx*Bi*b/(m_ref)+ T[1,j_t])/(1+Bi*H*dx)))#robins bc (two point forward)
        T[0, j_t] = round(((2*n*dx*Bi*b/(m_ref)+ 4*T[1,j_t] -T[2,j_t])/(3+2*Bi*H*dx)))#robins bc(three point forward)
        #T[0, j_t] = u_D*n
    s_vector[j_t] = s
    
    for j_x in range(N_x):
            if T[j_x, j_t] < 0:
                sign = -1
            else:
                sign = 1
            for j in range(sign*T[j_x, j_t]):
                p= 2*round(random.uniform(0, 1))-1 # =+-1, with P(+1)=P(-1)=1/2.
                if j_x+p > 0 and j_x + p <= j_s  and j_x <= N_x-1: # A walker move if it has not reached the boundaries.
                    T[j_x +p, j_t+1] = T[j_x+p, j_t+1] + 1*sign
                    #print('T=', T)
                elif j_x + p == j_s+1: 
                    ds[j_t] = dt*A_0*(T[j_s, j_t] - (K/x_ref)*s/(m_ref)) #i added to check
                    P = (1/n)*np.sqrt(2/np.pi)*A_0*(T[j_s, j_t]*sign - (K/x_ref)*s/(m_ref))*np.sqrt(np.pi*dt) 
                    r = random.uniform(0,1) 
                    if r<P:
                        T[j_s , j_t+1] = T[j_s, j_t+1] +1
                        s = s + ds[j_t]/n*sign # when walker is on boundary, boundary is increased
                    else:
                        T[j_s-1, j_t+1] = T[j_s-1, j_t+1] + 1*sign 
                    j_s =   floor(s/dx)
    j_t = j_t +1
T = T/n #divide by the factor n
end_time = time.time()
print('Computation time =', (end_time-start_time)/60, 'minute(s)')


#for plotting purpose only (to run compare with fem, we first need to rub fem code)

x = np.linspace(0, (N_x-1)*dx, N_x) #space grid points
y = np.linspace(0, (N_t-1)*dt, N_t) #time grid points
#print('x=', x)
#print('y=', len(y))
X, Y = np.meshgrid(x, y)
t_vector = y[:-1]
t_d = t_vector 
ss_vector = s_vector[:-1]
fig = plt.figure(dpi=500)
#for i in range(len(Dt)):
#plt.plot(t_d[:][::time_refine], SS[0], 'r-') # random walk solution
plt.plot(t_d[:], ss_vector, 'r--', linewidth = 2) # random walk solution
#plt.plot(t_d[:], S_T[:len(t_d)], 'g-', linewidth = 2) #finite element solution
plt.plot(t_d[:], St[::NN], 'g-', linewidth = 2) #finite element solution
fontsize = 15
fontweight = 'normal'
fontproperties = {'weight' : fontweight, 'size' : fontsize}
#plt.axis([-0.0000001, np.max(t_d[:])+0.0000001, 0, np.max(S_T[:len(t_d)])+0.001])
plt.legend([r"RWM $(n=100)$",r"FEM"], loc='lower right', frameon=False) # loc='upper left', frameon=False
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel(r'$\tau$', fontsize= fontsize, fontweight=fontweight)
plt.ylabel(r'$h(\tau)$', fontsize= fontsize, fontweight=fontweight)
plt.show()

###PLOT CONCENTRATON PROFILE
fig = plt.figure(dpi=500)
plt.plot(x, T[:, int((1/NN)*T_dim[2])], 'r-', linewidth = 2) #random walk solution
plt.plot(md[T_dim[2], :], m[T_dim[2],:-1], 'g--' ,linewidth=2) #FEM
fontsize = 15
fontweight = 'normal'
fontproperties = {'weight' : fontweight, 'size' : fontsize}
plt.legend([r"RWM (n=100)",r"FEM"], loc='upper right', frameon=False) # loc='upper left', frameon=False
#plt.grid()
plt.xlabel('$z$', fontsize= fontsize, fontweight=fontweight)
plt.ylabel(r'$u(\tau, z)$', fontsize= fontsize, fontweight=fontweight)
plt.axis([-0.00005, 0.026, -0.02, 10.2])
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.plot()
plt.show()
#np.savez('mv_front_robin_dt_5e08_n1k_two_point.npz', ss_vector )
#np.savez('con_profile_robin_dt_5e08_n1k_two_point.npz', T)
